In [1]:
import pickle
import csv
from pprint import pprint as myprint
from bs4 import BeautifulSoup as bs
import thulac

In [2]:
# html_file = open(r"../tmp/html.txt","r",encoding="utf-8")
# html_string = html_file.read()
# html_file.close()
# soup = bs(html_string,"html5lib")
# root = soup.html.body
# print(root.prettify())

再次读取的时候，发现原有的层次结构被破化了,应该考虑别的方法存储BeautifulSoup对象而非使用字符串，另一种思路是继续在bigram2html文件中写下去
Python对象的序列化可以使用pickle模块来实现

In [3]:
html_file_b = open(r"../tmp/html.dat","rb")
soup = pickle.load(html_file_b)
root = soup.html.body

恢复二进制文件到对象

In [4]:
for item in list(root.descendants)[:20]:
    if isinstance(item,str):
        continue
    print(item.name,item.parent.name)

h1 body
h2 h1
h3 h2
h3 h2
h4 h3
h4 h3
h4 h3
h4 h3
h4 h3
h4 h3


此处遍历的顺序是无所谓的，但是BeautifulSoup按照预期的顺序遍历了，也就是创建时的顺序

In [5]:
triple_list = []
course_name = "信息资源建设"
verb = "sub-concept"
i = 1
for item in root.descendants:
    if isinstance(item,str):
        continue
    sbj = item.contents[0]
    if item.parent.name == "body":
        obj = course_name
    else:
        obj = item.parent.contents[0]
    triple_list.append((sbj,verb,obj))
myprint(triple_list[:20])

[('信息、信息媒体与信息资源', 'sub-concept', '信息资源建设'),
 ('信息概述', 'sub-concept', '信息、信息媒体与信息资源'),
 ('信息的定义从广义上理解，信息可以被认为是物质的一种属性，是物质存在方式和运动规律与特点的表现形式。从狭义上理解，信息是一种消息、信号、数据或资料，是用来消除不定性的东西。',
  'sub-concept',
  '信息概述'),
 ('信息的特征', 'sub-concept', '信息概述'),
 ('真实性', 'sub-concept', '信息的特征'),
 ('知识性', 'sub-concept', '信息的特征'),
 ('指向性', 'sub-concept', '信息的特征'),
 ('共享性', 'sub-concept', '信息的特征'),
 ('无限性', 'sub-concept', '信息的特征'),
 ('时效性', 'sub-concept', '信息的特征'),
 ('依附性', 'sub-concept', '信息的特征'),
 ('信息的种类', 'sub-concept', '信息概述'),
 ('按信息的产生和作用机制划分，有自然信息和社会信息两大类；', 'sub-concept', '信息的种类'),
 ('按信息是否被人感知来划分，有显性信息和隐性信息两大类；', 'sub-concept', '信息的种类'),
 ('按信息的运动状态来划分，有动态信息和静态信息两大类；', 'sub-concept', '信息的种类'),
 ('按信息的存在和接受情况划分，有直接信息和间接信息两大类。', 'sub-concept', '信息的种类'),
 ('信息的功能与作用', 'sub-concept', '信息概述'),
 ('维系社会存在与发展', 'sub-concept', '信息的功能与作用'),
 ('社会经济发展的动力', 'sub-concept', '信息的功能与作用'),
 ('科学决策、科学研究和生产建设的参谋和依据', 'sub-concept', '信息的功能与作用')]


遍历html文档树的所有节点，并获得其父节点，子结点与父节点之间存在着语义上的被包含与包含关系

In [6]:
trigram_file = open(r"../tmp/trigram.csv","w",encoding="utf-8")
csv_writer = csv.writer(trigram_file)
for item in triple_list:
    csv_writer.writerow(item)
trigram_file.close()

将结果写在csv文件中，同样暂时没有什么用，只是保存当前成果

In [7]:
stop_set = set()
stop_list_file_1 = open(r"../stoplist/哈工大停用词表.txt","r",encoding="utf8")
stop_list_file_2 = open(r"../stoplist/百度停用词表.txt","r",encoding="utf8")
stop_list_file_3 = open(r"../stoplist/中文停用词表.txt","r",encoding="utf8")
stop_list_file_4 = open(r"../stoplist/四川大学机器智能实验室停用词库.txt","r",encoding="utf8")
for word in stop_list_file_1.readlines():
    stop_set.add(word.strip())
for word in stop_list_file_2.readlines():
    stop_set.add(word.strip())
for word in stop_list_file_3.readlines():
    stop_set.add(word.strip())
for word in stop_list_file_4.readlines():
    stop_set.add(word.strip())
print(list(stop_set)[:100])

['', 'seven', '每时每刻', 'except', '由', 'an', 'were', '要求', '与此同时', 'enough', 'was', '宁肯', '积极', '因为', '由于', '怎', '成为', '以便', 'described', '凭借', '不力', '某些', '故', '八成', '适当', 'noone', '然后', 'now', 'saw', '争取', 'for', 'selves', 'despite', '深入', 'associated', '重要', '绝不', '当时', 'let', '明显', '末##末', '以下', '［②⑤］', '遵照', '范围', 'getting', '＝', '呜', 'ourselves', 'three', '不尽', '平素', 'comes', '嘻', '巨大', '总而言之', '万一', "aren't", '下列', '每个', '巩固', '宁', '③', '亲身', 'ex', '故意', '运用', '又', 'having', '达到', '(', '下来', '什麽', 'at', '去', '截至', '［⑦］', '上述', '尽如人意', '以至', 'useful', '重大', 'our', '独自', 'done', '果然', '亦', 'follows', '凡是', '这个', '故此', '倘然', '及时', '若', 'through', '巴巴', 'furthermore', '@', '与其说', '本']


准备一个停用词表

In [8]:
thu = thulac.thulac(seg_only=True)
s_string,verb,o_string = triple_list[2]
# s_list = thu.cut(s_string,text=False)
s_list = [word for word,pos in thu.cut(s_string,text=False) if word not in stop_set]
print(s_list)

Model loaded succeed
['信息', '定义', '广义', '理解', '信息', '物质', '属性', '物质', '方式', '运动', '规律', '表现', '形式', '狭义', '理解', '信息', '消息', '信号', '数据', '资料', '消除', '定性', '东西']


测试分词的功能是否可用

In [9]:
triple_set = set()
thu = thulac.thulac(seg_only=True)
for s_string,v_string,o_string in triple_list:
    s_list = [word for word,pos in thu.cut(s_string,text=False) if word not in stop_set]
    o_list = [word for word,pos in thu.cut(o_string,text=False) if word not in stop_set]
    for s in s_list:
        for o in o_list:
            triple_set.add((s,verb,o))    
myprint(list(triple_set)[:20])

Model loaded succeed
[('转换', 'sub-concept', '资源'),
 ('资源', 'sub-concept', '客观'),
 ('共', 'sub-concept', '再版'),
 ('随意性', 'sub-concept', '称'),
 ('纸质', 'sub-concept', '建设'),
 ('含有', 'sub-concept', '般'),
 ('称', 'sub-concept', '印刷'),
 ('两', 'sub-concept', '种类'),
 ('单位', 'sub-concept', '续'),
 ('内容', 'sub-concept', '客观'),
 ('表现', 'sub-concept', '分'),
 ('信息源', 'sub-concept', '类'),
 ('流通', 'sub-concept', '手段'),
 ('实物', 'sub-concept', '分'),
 ('转换', 'sub-concept', '看法'),
 ('记载', 'sub-concept', '固定'),
 ('被摘率', 'sub-concept', '纸张'),
 ('国防部', 'sub-concept', '资源'),
 ('浓度', 'sub-concept', '信息'),
 ('记载', 'sub-concept', '期刊')]


至此，一篇文档就由文本转换成了三元组
浏览上面的结果，目前比较难判断处理的效果，但是从一些元组中可以明显看到可用的逻辑
对于并不很严肃的课件能得到这样的结果是满意的

In [10]:
trigram_file_update = open(r"../tmp/trigram_update.csv","w",encoding="utf-8")
csv_writer = csv.writer(trigram_file_update)
for item in triple_set:
    csv_writer.writerow(item)
trigram_file_update.close()

写入三元组列表到csv文件中备用

In [11]:
trigram_b = open(r"../tmp/trigram.dat","wb")
pickle.dump(triple_set,trigram_b)
trigram_b.close()

保存三元组列表到二进制文件